 # Data Cleaning and Transformation (ETL) for Credit Default Prediction
 
 This notebook is the first step in our data analysis pipeline, focusing on the Extract, Transform, Load (ETL) process for the credit card default dataset. The primary goal is to prepare the raw data for subsequent exploratory data analysis (EDA) and machine learning modeling.
 
 The key steps performed in this notebook include:
 
 -   **Loading the Dataset:** Reading the raw data into a Pandas DataFrame.
 -   **Initial Data Exploration:** Understanding the dataset's structure, dimensions, and initial data types.
 -   **Handling Missing/Invalid Values:** Checking for and addressing any data quality issues such as nulls, duplicates, or unexpected negative values.
 -   **Renaming Columns:** Improving readability and consistency of column names.
 -   **Mapping Categorical Values:** Converting numerical codes in categorical features into more descriptive, human-readable labels.
 -   **Converting Data Types:** Ensuring all columns have appropriate data types for efficient analysis.
 -   **Saving Cleaned Data:** Exporting the processed data for use in the next phases of the project.


##  Load the Dataset and Explore Its Structure

 Before any cleaning or analysis, it's crucial to understand the raw data. This step involves loading the dataset and performing initial inspections to grasp its fundamental structure.
 
 In this section, we:
 -   Load the dataset into a DataFrame using Pandas.
 -   Display the first few rows to preview the data content.
 -   Examine the overall dimensions (number of rows and columns) of the dataset.
 -   Review the column names and their inferred data types to identify any immediate issues.


In [31]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
os.makedirs("../assets/plots", exist_ok=True) # Changed path to reflect a common 'assets/plots' structure


In [32]:
# Load the dataset from the specified path
df = pd.read_csv("../data/inputs/raw/uci_credit_card.csv")
print("Dataset loaded successfully.")
df.head()

Dataset loaded successfully.


,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default.payment.next.month
0,1,20000.0,2,2,1,24,2,2,-1,-1,...,0.0,0.0,0.0,0.0,689.0,0.0,0.0,0.0,0.0,1
1,2,120000.0,2,2,2,26,-1,2,0,0,...,3272.0,3455.0,3261.0,0.0,1000.0,1000.0,1000.0,0.0,2000.0,1
2,3,90000.0,2,2,2,34,0,0,0,0,...,14331.0,14948.0,15549.0,1518.0,1500.0,1000.0,1000.0,1000.0,5000.0,0
3,4,50000.0,2,2,1,37,0,0,0,0,...,28314.0,28959.0,29547.0,2000.0,2019.0,1200.0,1100.0,1069.0,1000.0,0
4,5,50000.0,1,2,1,57,-1,0,-1,0,...,20940.0,19146.0,19131.0,2000.0,36681.0,10000.0,9000.0,689.0,679.0,0


In [33]:
# Display the shape of the dataset (number of rows, number of columns)
df.shape

(30000, 25)

In [34]:
# Check the data types of each column
df.dtypes

ID                              int64
LIMIT_BAL                     float64
SEX                             int64
EDUCATION                       int64
MARRIAGE                        int64
AGE                             int64
PAY_0                           int64
PAY_2                           int64
PAY_3                           int64
PAY_4                           int64
PAY_5                           int64
PAY_6                           int64
BILL_AMT1                     float64
BILL_AMT2                     float64
BILL_AMT3                     float64
BILL_AMT4                     float64
BILL_AMT5                     float64
BILL_AMT6                     float64
PAY_AMT1                      float64
PAY_AMT2                      float64
PAY_AMT3                      float64
PAY_AMT4                      float64
PAY_AMT5                      float64
PAY_AMT6                      float64
default.payment.next.month      int64
dtype: object

### Initial Data Overview
 
 The dataset consists of **30,000 rows** and **25 columns**.
 
 * The `ID` column is a unique identifier for each client.
 * Most features are numerical (`int64`, `float64`), including credit limits, age, and various financial amounts.
 * Categorical features like `SEX`, `EDUCATION`, and `MARRIAGE` are currently represented as integers, which will require mapping to more descriptive labels.
 * The target variable, `default.payment.next.month`, is also an integer (0 or 1).
 
 This initial check confirms the data has been loaded correctly and highlights areas that need further transformation.


##  Handle Missing or Invalid Values
 
 After understanding the basic structure, the next critical step is to assess the data quality by checking for missing or invalid entries. This ensures the integrity of our dataset before any analytical or modeling tasks.
 
 In this section, we:
 -   Check for the presence and count of any missing (null) values across all columns.
 -   Identify and count duplicate rows, as these can bias analysis.
 -   Verify for unexpected negative values in numerical columns, which might indicate data entry errors or unusual financial behavior.


In [35]:
# Check for missing values across all columns and sum them up
df.isnull().sum()

ID                            0
LIMIT_BAL                     0
SEX                           0
EDUCATION                     0
MARRIAGE                      0
AGE                           0
PAY_0                         0
PAY_2                         0
PAY_3                         0
PAY_4                         0
PAY_5                         0
PAY_6                         0
BILL_AMT1                     0
BILL_AMT2                     0
BILL_AMT3                     0
BILL_AMT4                     0
BILL_AMT5                     0
BILL_AMT6                     0
PAY_AMT1                      0
PAY_AMT2                      0
PAY_AMT3                      0
PAY_AMT4                      0
PAY_AMT5                      0
PAY_AMT6                      0
default.payment.next.month    0
dtype: int64

In [36]:
# Check for duplicate rows in the dataset
df.duplicated().sum()

0

In [37]:
# List numeric columns
numeric_cols = df.select_dtypes(include=["float64", "int64"]).columns

# We'll check all numerical columns and interpret contextually.
df[numeric_cols].lt(0).sum()


ID                                0
LIMIT_BAL                         0
SEX                               0
EDUCATION                         0
MARRIAGE                          0
AGE                               0
PAY_0                          8445
PAY_2                          9832
PAY_3                         10023
PAY_4                         10035
PAY_5                         10085
PAY_6                         10635
BILL_AMT1                       590
BILL_AMT2                       669
BILL_AMT3                       655
BILL_AMT4                       675
BILL_AMT5                       655
BILL_AMT6                       688
PAY_AMT1                          0
PAY_AMT2                          0
PAY_AMT3                          0
PAY_AMT4                          0
PAY_AMT5                          0
PAY_AMT6                          0
default.payment.next.month        0
dtype: int64

Interpretation of Negative Values:
 - 'PAY_0' to 'PAY_6' columns (payment status) can legitimately have -1 (paid on time) or -2 (no consumption/revolving credit).
 - 'BILL_AMT' and 'PAY_AMT' columns (bill and payment amounts) can also legitimately have negative values, indicating credit balances or refunds.
 - 'LIMIT_BAL', 'AGE', 'ID', and 'default.payment.next.month' should ideally not have negative values.
 The current output shows negative values primarily in PAY_x, BILL_AMTx, and PAY_AMTx, which are expected based on the dataset's documentation.


##  Rename Columns and Map Categorical Values

 To enhance the readability and usability of our dataset, this step focuses on standardizing column names and transforming coded categorical values into understandable labels.
 
 Specifically, we will:
 1.  **Standardize Column Names**: Convert all column names to lowercase, remove leading/trailing spaces, and replace spaces with underscores for consistency and ease of access.
 2.  **Descriptive Renaming**: Replace cryptic column names (e.g., `PAY_0`, `BILL_AMT1`) with more intuitive and descriptive names (e.g., `pay_delay_sep`, `bill_amount_sep`).
 3.  **Map Categorical Values**: Convert numerical codes in `sex`, `education`, and `marriage` into meaningful string labels (e.g., 1 for `sex` becomes "male").


In [38]:
# Convert all column names to lowercase, strip leading/trailing spaces, and replace spaces with underscores
df.columns = df.columns.str.lower().str.strip().str.replace(" ", "_")

In [39]:
# Rename the target column for clarity and consistency
df.rename(columns={"default.payment.next.month": "default"}, inplace=True)

In [40]:
# Rename payment status, bill amount, and payment amount columns for better readability
df.rename(columns={
    "pay_0": "pay_delay_sep",
    "pay_2": "pay_delay_aug",
    "pay_3": "pay_delay_jul",
    "pay_4": "pay_delay_jun",
    "pay_5": "pay_delay_may",
    "pay_6": "pay_delay_apr",
    "bill_amt1": "bill_amount_sep",
    "bill_amt2": "bill_amount_aug",
    "bill_amt3": "bill_amount_jul",
    "bill_amt4": "bill_amount_jun",
    "bill_amt5": "bill_amount_may",
    "bill_amt6": "bill_amount_apr",
    "pay_amt1": "payment_amount_sep",
    "pay_amt2": "payment_amount_aug",
    "pay_amt3": "payment_amount_jul",
    "pay_amt4": "payment_amount_jun",
    "pay_amt5": "payment_amount_may",
    "pay_amt6": "payment_amount_apr",
}, inplace=True)

### Understanding Payment Delay Codes
 
 The dataset includes several columns representing **repayment status** for different months, originally named `PAY_0`, `PAY_2`, ..., `PAY_6`, and now renamed to:
 
 -   `pay_delay_sep` (September)
 -   `pay_delay_aug` (August)
 -   `pay_delay_jul` (July)
 -   `pay_delay_jun` (June)
 -   `pay_delay_may` (May)
 -   `pay_delay_apr` (April)
 
 Each numerical value in these columns corresponds to a specific payment status, as detailed below:
 
 | Value | Meaning                        |
 | :---- | :----------------------------- |
 | -2    | No consumption (no balance)    |
 | -1    | Paid on time                   |
 | 0     | Duly paid or revolving credit  |
 | 1     | Payment delay for 1 month      |
 | 2     | Payment delay for 2 months     |
 | 3     | Payment delay for 3 months     |
 | 4     | Payment delay for 4 months     |
 | 5     | Payment delay for 5 months     |
 | 6     | Payment delay for 6 months     |
 | 7     | Payment delay for 7 months     |
 | 8     | Payment delay for 8 months     |
 | 9     | Payment delay for 9 months or more |
 
 Keeping these codes as numerical values is beneficial for statistical analysis and machine learning models, as the values represent an ordinal scale of increasing delinquency.


In [41]:
# Map numerical codes in categorical variables to descriptive string labels
df["sex"] = df["sex"].map({1: "male", 2: "female"})

df["education"] = df["education"].map({
    1: "graduate school",
    2: "university",
    3: "high school",
    4: "others",
    5: "unknown",
    6: "unknown"
})

df["marriage"] = df["marriage"].map({
    1: "married",
    2: "single",
    3: "others",
})

df.head()

,id,limit_bal,sex,education,marriage,age,pay_delay_sep,pay_delay_aug,pay_delay_jul,pay_delay_jun,...,bill_amount_jun,bill_amount_may,bill_amount_apr,payment_amount_sep,payment_amount_aug,payment_amount_jul,payment_amount_jun,payment_amount_may,payment_amount_apr,default
0,1,20000.0,female,university,married,24,2,2,-1,-1,...,0.0,0.0,0.0,0.0,689.0,0.0,0.0,0.0,0.0,1
1,2,120000.0,female,university,single,26,-1,2,0,0,...,3272.0,3455.0,3261.0,0.0,1000.0,1000.0,1000.0,0.0,2000.0,1
2,3,90000.0,female,university,single,34,0,0,0,0,...,14331.0,14948.0,15549.0,1518.0,1500.0,1000.0,1000.0,1000.0,5000.0,0
3,4,50000.0,female,university,married,37,0,0,0,0,...,28314.0,28959.0,29547.0,2000.0,2019.0,1200.0,1100.0,1069.0,1000.0,0
4,5,50000.0,male,university,married,57,-1,0,-1,0,...,20940.0,19146.0,19131.0,2000.0,36681.0,10000.0,9000.0,689.0,679.0,0


## 4. Convert Data Types
 
 To ensure efficient memory usage and correct behavior of statistical and machine learning functions, it's essential to assign appropriate data types to each column.
 
 In this step, we:
 -   Convert columns that represent distinct categories (like `sex`, `education`, `marriage`) to the `category` data type. This is more memory-efficient and explicitly tells Pandas and other libraries that these are categorical, not numerical, values.


In [42]:
# Identify columns that should be of 'category' data type
categorical_cols = ["sex", "education", "marriage"]

# Convert specified columns to 'category' type
df[categorical_cols] = df[categorical_cols].astype("category")

# Check data types after conversion
df.dtypes

id                       int64
limit_bal              float64
sex                   category
education             category
marriage              category
age                      int64
pay_delay_sep            int64
pay_delay_aug            int64
pay_delay_jul            int64
pay_delay_jun            int64
pay_delay_may            int64
pay_delay_apr            int64
bill_amount_sep        float64
bill_amount_aug        float64
bill_amount_jul        float64
bill_amount_jun        float64
bill_amount_may        float64
bill_amount_apr        float64
payment_amount_sep     float64
payment_amount_aug     float64
payment_amount_jul     float64
payment_amount_jun     float64
payment_amount_may     float64
payment_amount_apr     float64
default                  int64
dtype: object

## Save Cleaned Data
 
 The final step in the ETL process is to save the cleaned and transformed DataFrame. This ensures that the prepared data can be easily accessed for subsequent analysis phases (Exploratory Data Analysis and Machine Learning Modeling) without needing to re-run the entire cleaning script.


In [43]:
# Save the cleaned DataFrame to a new CSV file
# index=False prevents pandas from writing the DataFrame index as a column in the CSV
output_path = "../data/inputs/cleaned/cleaned_credit_data.csv"
df.to_csv(output_path, index=False)
print(f"Cleaned data saved successfully to: {output_path}")

Cleaned data saved successfully to: ../data/inputs/cleaned/cleaned_credit_data.csv
